# Validación de la clasificación del datasdet como violencia política de genero con el modelo RoBERTuito que está entrenado para clasificar tuits en positivos, negativos y neutros.


> Compara los tuits clasificados por medio del crowdsourcing como violencia política de género, con los tuits clasificados por modelo RoBERTuito como negativos.



Instalación de dependencias

In [1]:
!pip install pysentimiento

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd

Cargar Dataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
datos_C=pd.read_csv('/content/drive/MyDrive/ViolenciaPoíticaDeGénero/Transformación/data_clean_clasificado.csv')
datos_C.dropna(inplace = True)
datos_C.head(10)

,id,text,task
0,0,las glosas de muñoz que se quede para financia...,no violencia política de género
1,1,usted pavel un gran alcalde,no violencia política de género
2,2,dígame una ley que hacen desde la asamblea par...,no violencia política de género
3,3,el dinero de los afiliados al iess que desapar...,no violencia política de género
4,4,de este caso me estaba acordando,no violencia política de género
5,5,ojalá no sea la misma innovación de la roboluc...,no violencia política de género
6,6,habló la que se ganó tres veces la lotería,no violencia política de género
7,7,recuerda ese argentino que le dieron tremendo ...,no violencia política de género
8,8,pierrina calladita tu hermano es el que ha sac...,violencia política de género
9,9,siempre sido asi los únicos que se llenan los ...,no violencia política de género


Cargar Modelo

In [6]:
from transformers import AutoModelForSequenceClassification
model_name = "pysentimiento/robertuito-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)

Some weights of the model checkpoint at pysentimiento/robertuito-base-cased were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pysentimiento/robertuito-base-cased and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight

Crear analizador

In [7]:
from pysentimiento import create_analyzer
analyzer = create_analyzer(task="sentiment", lang="es")

Clasificar con el Modelo la columna text del dataset

In [8]:
labels = []
for i in datos_C['text']:
  labels.append(analyzer.predict(i))

In [9]:
data_labels = pd.DataFrame({'id': datos_C['id'], 'text': datos_C['text'],'task': datos_C['task'], 'labels': [i.output for i in labels]})
data_labels.head(10)

,id,text,task,labels
0,0,las glosas de muñoz que se quede para financia...,no violencia política de género,NEG
1,1,usted pavel un gran alcalde,no violencia política de género,POS
2,2,dígame una ley que hacen desde la asamblea par...,no violencia política de género,NEG
3,3,el dinero de los afiliados al iess que desapar...,no violencia política de género,NEU
4,4,de este caso me estaba acordando,no violencia política de género,NEU
5,5,ojalá no sea la misma innovación de la roboluc...,no violencia política de género,NEG
6,6,habló la que se ganó tres veces la lotería,no violencia política de género,NEU
7,7,recuerda ese argentino que le dieron tremendo ...,no violencia política de género,NEG
8,8,pierrina calladita tu hermano es el que ha sac...,violencia política de género,NEG
9,9,siempre sido asi los únicos que se llenan los ...,no violencia política de género,NEG


Comparar los tuits clasificados como violencia política de género, con los tuits clasificados por modelo RoBERTuito como negativos.

In [10]:
compare = []
for i in range(len(data_labels)):
  if data_labels['labels'][i] == 'NEG' and data_labels['task'][i][:9] == 'violencia':
    compare.append(1)
  else:
    compare.append(0)

In [11]:
df = pd.DataFrame({'id': datos_C['id'], 'text': datos_C['text'], 'task': datos_C['task'], 'labels': [i.output for i in labels], 'compare': compare})
df.head(20)

,id,text,task,labels,compare
0,0,las glosas de muñoz que se quede para financia...,no violencia política de género,NEG,0
1,1,usted pavel un gran alcalde,no violencia política de género,POS,0
2,2,dígame una ley que hacen desde la asamblea par...,no violencia política de género,NEG,0
3,3,el dinero de los afiliados al iess que desapar...,no violencia política de género,NEU,0
4,4,de este caso me estaba acordando,no violencia política de género,NEU,0
5,5,ojalá no sea la misma innovación de la roboluc...,no violencia política de género,NEG,0
6,6,habló la que se ganó tres veces la lotería,no violencia política de género,NEU,0
7,7,recuerda ese argentino que le dieron tremendo ...,no violencia política de género,NEG,0
8,8,pierrina calladita tu hermano es el que ha sac...,violencia política de género,NEG,1
9,9,siempre sido asi los únicos que se llenan los ...,no violencia política de género,NEG,0


In [12]:
for i in range(0, len(df)):
  if df['labels'][i] == 'POS' and df['task'][i][:9] == 'violencia':
    print(df['id'][i],df['text'][i],df['labels'][i], ' ', df['task'][i])

2718 honrada buena broma POS   violencia política de género
3231 felicitaciones le ha dicho una gran verdad esta borrega sanduchera POS   violencia política de género
3269 delincuente seguro esta obra no tiene sobreprecio tiene material de calidad POS   violencia política de género
4122 delira POS   violencia política de género
4256 esta borrega si que es una bestia POS   violencia política de género
8857 juntos podemos robar más POS   violencia política de género
9183 que viva eternamente sin lacras correistas POS   violencia política de género
9621 guayaquil es una ciudad hermosa siempre borrega no solo en tiempos de política POS   violencia política de género
10262 que lindo es entrar ver cómo te arremangan cada vez rebuznas POS   violencia política de género
16344 si muy bien testaferra POS   violencia política de género
17120 buen dicho mi oveja tetona POS   violencia política de género
17292 entre delincuentes se apoyan qué tiernos POS   violencia política de género
17749 el prov

In [13]:
data_labels['task'].value_counts()

no violencia política de género    16045
violencia política de género        3610
Name: task, dtype: int64

In [14]:
df['compare'].value_counts()

0    16353
1     3302
Name: compare, dtype: int64

In [15]:
print((3302*100)/3610)

91.46814404432133


In [ ]:
df.to_csv('/content/drive/MyDrive/ViolenciaPoíticaDeGénero/Transformación/validacion.csv',index=False)